# <center> <font size="+10"> <bold> ETL </bold> </font> </center>

In [1]:
#Import dependencies
import pandas as pd
from sqlalchemy import create_engine

# Extract

In [2]:
#Read csv obtained from Mexico's Government site
CP_CDMX_df = pd.read_csv("Resources/CodigosPostales.csv")
CP_CDMX_df.head()

,Código Postal,Estado,Municipio,Ciudad,Tipo de Asentamiento,Asentamiento,Clave de Oficina
0,9830,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,Lomas El Manto,9821
1,9830,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,Los Ángeles,9821
2,9830,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,Paraje San Juan,9821
3,9837,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,San Miguel 8va. Ampliación,9821
4,9838,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,Plan de Iguala,9821


In [3]:
#Read csv obtained from Kaggle and last updated in October 2019.

Airbnb_CDMX_df = pd.read_csv("Resources/AirbnbCDMX.csv")
Airbnb_CDMX_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,14714,Private room c/ bathroom . Colonia Juarez.,57785,Diego,NaN,Cuauhtémoc,19.43035,-99.15511,Private room,478,2,0,NaN,NaN,2,327
1,22787,"Sunny suite w/ queen size bed, inside boutique...",87973,Diego,NaN,Cuauhtémoc,19.44076,-99.16324,Private room,1969,1,57,2019-05-27,0.52,9,355
2,33681,Couple of Rooms,145672,Edubiel,NaN,Tlalpan,19.27215,-99.21848,Private room,1740,1,0,NaN,NaN,1,365
3,35797,Villa Dante,153786,Dici,NaN,Cuajimalpa de Morelos,19.38399,-99.27335,Entire home/apt,3823,1,0,NaN,NaN,2,363
4,44616,CONDESA HAUS B&B,196253,Condesa Haus Bed & Breakfast CDMX,NaN,Cuauhtémoc,19.41006,-99.17645,Private room,1893,1,39,2019-05-02,0.42,10,334


In [4]:
#Read csv obtained from Kaggle and last updated in February 2017.

Starbucksdirectory_df = pd.read_csv("Resources/Starbucksdirectory.csv")
Starbucksdirectory_df.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


# Transform

In [5]:
# Change table language to English

CP_CDXM_df_e = CP_CDMX_df.rename(columns = {'Código Postal':'postcode',
                            'Estado':'state_name',
                            'Municipio':'municipality_name',
                            'Ciudad':'city',
                            'Tipo de Asentamiento':'Settlement Type',
                            'Asentamiento':'settlement',
                            'Clave de Oficina':'Office Code'})
CP_CDXM_df_e.head()

,postcode,state_name,municipality_name,city,Settlement Type,settlement,Office Code
0,9830,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,Lomas El Manto,9821
1,9830,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,Los Ángeles,9821
2,9830,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,Paraje San Juan,9821
3,9837,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,San Miguel 8va. Ampliación,9821
4,9838,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,Plan de Iguala,9821


In [6]:
# Select only the columns we need
new_postal_codes = CP_CDXM_df_e[['postcode', 'state_name', 'municipality_name', 'city', 'settlement']].copy()
new_postal_codes['id_pc'] = [x for x in range(len(new_postal_codes))]
new_postal_codes.head()

,postcode,state_name,municipality_name,city,settlement,id_pc
0,9830,Ciudad de México,Iztapalapa,Ciudad de México,Lomas El Manto,0
1,9830,Ciudad de México,Iztapalapa,Ciudad de México,Los Ángeles,1
2,9830,Ciudad de México,Iztapalapa,Ciudad de México,Paraje San Juan,2
3,9837,Ciudad de México,Iztapalapa,Ciudad de México,San Miguel 8va. Ampliación,3
4,9838,Ciudad de México,Iztapalapa,Ciudad de México,Plan de Iguala,4


In [7]:
#Select only the columns we need from Airbnb file.

new_Airbnb = Airbnb_CDMX_df[['name', 'neighbourhood', 'latitude', 'longitude', 'room_type']].copy()
new_Airbnb['id_air'] = [x for x in range(len(new_Airbnb))]
new_Airbnb.head()

,name,neighbourhood,latitude,longitude,room_type,id_air
0,Private room c/ bathroom . Colonia Juarez.,Cuauhtémoc,19.43035,-99.15511,Private room,0
1,"Sunny suite w/ queen size bed, inside boutique...",Cuauhtémoc,19.44076,-99.16324,Private room,1
2,Couple of Rooms,Tlalpan,19.27215,-99.21848,Private room,2
3,Villa Dante,Cuajimalpa de Morelos,19.38399,-99.27335,Entire home/apt,3
4,CONDESA HAUS B&B,Cuauhtémoc,19.41006,-99.17645,Private room,4


In [8]:
Starbucksdirectory_df=Starbucksdirectory_df.rename(columns = {'Store Name':'store_name',
                                                              'Street Address':'street_address',
                                                              'State/Province':'state_name',
                                                              'Country':'country_name',
                                                              'Postcode':'postcode',
                                                              'Longitude':'longitude',
                                                              'Latitude':'latitude'})

In [9]:
#Select only the columns we need from Starbucks file.

new_Starbucks = Starbucksdirectory_df [['store_name', 'street_address', 'state_name', 'country_name', 'postcode', 'longitude', 'latitude']].copy()
new_Starbucks.head()

,store_name,street_address,state_name,country_name,postcode,longitude,latitude
0,"Meritxell, 96","Av. Meritxell, 96",7,AD,AD500,1.53,42.51
1,Ajman Drive Thru,"1 Street 69, Al Jarf",AJ,AE,NaN,55.47,25.42
2,Dana Mall,Sheikh Khalifa Bin Zayed St.,AJ,AE,NaN,55.47,25.39
3,Twofour 54,Al Salam Street,AZ,AE,NaN,54.38,24.48
4,Al Ain Tower,"Khaldiya Area, Abu Dhabi Island",AZ,AE,NaN,54.54,24.51


In [10]:
#Filter Starbucks data for Mexico City only

Starbucks_MX = new_Starbucks.loc [new_Starbucks['country_name']== 'MX']
Starbucks_MX = new_Starbucks.loc [new_Starbucks['state_name'] == 'DIF']

Starbucks_MX['id_starbucks'] = [x for x in range(len(Starbucks_MX))]

Starbucks_MX.head()

C:\Users\Joshua Castillo\AppData\Local\conda\conda\envs\actumlogos-gpu\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,store_name,street_address,state_name,country_name,postcode,longitude,latitude,id_starbucks
9184,STIM,"de la Industria Militar 1296,, Lomas de Chamiz...",DIF,MX,5129,-99.26,19.39,0
9185,EMERSON,"Homero n.503, Col. Chapultepec Morales, Del. M...",DIF,MX,11570,-99.19,19.44,1
9186,Narvarte,"Palanque s/n, Col. Narvarte",DIF,MX,1030,-99.15,19.40,2
9187,Pasaje San Jeronimo,AV SAN JERONIMO 819 COL SAN JERONIMO,DIF,MX,10200,-99.23,19.32,3
9188,RENATO LEDUC,RENATO LEDUC 224 COL TORIELLO GUERRA,DIF,MX,14050,-99.16,19.30,4


# Load

In [11]:
rds_connection_string = "postgres:Joshua007@localhost:5432/ETL_Project"
engine_text = "postgresql://" + rds_connection_string
engine = create_engine(engine_text)

In [12]:
engine.table_names()

['Postcodes',
 'Lol',
 'starbucks',
 'postcodes',
 'airbnb',
 'joinedtable2',
 'joinedtable3']

In [13]:
Starbucks_MX.to_sql(name='starbucks', con=engine, if_exists='append', index=False)
pd.read_sql_query('select * from starbucks', con=engine).head()

,id_starbucks,store_name,street_address,state_name,country_name,postcode,longitude,latitude
0,0,STIM,"de la Industria Militar 1296,, Lomas de Chamiz...",DIF,MX,5129.0,-99.26,19.39
1,1,EMERSON,"Homero n.503, Col. Chapultepec Morales, Del. M...",DIF,MX,11570.0,-99.19,19.44
2,2,Narvarte,"Palanque s/n, Col. Narvarte",DIF,MX,1030.0,-99.15,19.40
3,3,Pasaje San Jeronimo,AV SAN JERONIMO 819 COL SAN JERONIMO,DIF,MX,10200.0,-99.23,19.32
4,4,RENATO LEDUC,RENATO LEDUC 224 COL TORIELLO GUERRA,DIF,MX,14050.0,-99.16,19.30


In [14]:
new_Airbnb.to_sql(name='airbnb', con=engine, if_exists='append', index=False)
pd.read_sql_query('select * from airbnb', con=engine).head()

,id_air,name,neighbourhood,latitude,longitude,room_type,latitude_aprox,longitude_aprox
0,0,Private room c/ bathroom . Colonia Juarez.,Cuauhtémoc,19.43035,-99.15511,Private room,None,None
1,1,"Sunny suite w/ queen size bed, inside boutique...",Cuauhtémoc,19.44076,-99.16324,Private room,None,None
2,2,Couple of Rooms,Tlalpan,19.27215,-99.21848,Private room,None,None
3,3,Villa Dante,Cuajimalpa de Morelos,19.38399,-99.27335,Entire home/apt,None,None
4,4,CONDESA HAUS B&B,Cuauhtémoc,19.41006,-99.17645,Private room,None,None


In [15]:
new_postal_codes.to_sql(name='postcodes', con=engine, if_exists='append', index=False)
pd.read_sql_query('select * from Postcodes', con=engine).head()

,id_pc,postcode,state_name,municipality_name,city,settlement
0,0,9830.0,Ciudad de México,Iztapalapa,Ciudad de México,Lomas El Manto
1,1,9830.0,Ciudad de México,Iztapalapa,Ciudad de México,Los Ángeles
2,2,9830.0,Ciudad de México,Iztapalapa,Ciudad de México,Paraje San Juan
3,3,9837.0,Ciudad de México,Iztapalapa,Ciudad de México,San Miguel 8va. Ampliación
4,4,9838.0,Ciudad de México,Iztapalapa,Ciudad de México,Plan de Iguala
